In [313]:
#Sample graph representation
"""
graph = {
    1: [2,5],
    2: [3,5],
    3: [6, 7],
    4: [1,5],
    5: [6],
    6: [2],
    7: [],
}
"""

graph = {
    'L': ['R'],
    'R': ['D','T'],
    'B': ['T'],
    'D': [],
    'T': ['Z'],
    'Z': [],
}

def disconnected(G, X, Y):
   #bfs path search for some node in X to some node in Y

    visited = []
    stack = []
    stack.append(X[0])
    while len(stack) != 0:
        node = stack.pop(0)
        if node == Y[0]:
            return False
        visited.append(node)
        for x in G[node]:
            if x not in visited:
                stack.append(x)
            
    return True
            
                

def d_seperation(G, X, Y, Z):
    
    #pruning
    # 1. Remove leaf nodes not in X + Y + Z
    union = X + Y + Z
    for leaf in G.copy():
        if len(G[leaf]) == 0 and leaf not in union:
            del G[leaf]
            # Delete incoming edges to previously removed leaf
            for node in G.keys():
                try:
                    G[node].remove(leaf)
                except ValueError:
                    pass  #leaf not in that node
                
    #2. remove outgoing edges from Z nodes           
    for node in Z:
        G[node] = []
    
              
    return disconnected(G,X,Y)

a = d_seperation(graph, ['L'], ['B'], ['T'])
print(a)

True


In [124]:
#Get data from file, learn parameters
from collections import OrderedDict as odict
import pandas as pd
from itertools import product, combinations
import numpy as np
from tabulate import tabulate


#================= Code from Week 3 Tutorial ====================================


def allEqualThisIndex(dict_of_arrays, **fixed_vars):
    """
    Helper function to create a boolean index vector into a tabular data structure,
    such that we return True only for rows of the table where, e.g.
    column_a=fixed_vars['column_a'] and column_b=fixed_vars['column_b'].
    
    This is a simple task, but it's not *quite* obvious
    for various obscure technical reasons.
    
    It is perhaps best explained by an example.
    
    >>> all_equal_this_index(
    ...    {'X': [1, 1, 0], Y: [1, 0, 1]},
    ...    X=1,
    ...    Y=1
    ... )
    [True, False, False]
    """
    # base index is a boolean vector, everywhere true
    first_array = dict_of_arrays[list(dict_of_arrays.keys())[0]]
    index = np.ones_like(first_array, dtype=np.bool_)
    for var_name, var_val in fixed_vars.items():
        index = index & (np.asarray(dict_of_arrays[var_name])==var_val)
    return index



def printFactor(f):
    """
    argument 
    `f`, a factor to print on screen
    """
    # Create a empty list that we will fill in with the probability table entries
    table = list()
    
    # Iterate over all keys and probability values in the table
    for key, item in f['table'].items():
        # Convert the tuple to a list to be able to manipulate it
        k = list(key)
        # Append the probability value to the list with key values
        k.append(item)
        # Append an entire row to the table
        table.append(k)
    # dom is used as table header. We need it converted to list
    dom = list(f['dom'])
    # Append a 'Pr' to indicate the probabity column
    dom.append('Pr')
    print(tabulate(table,headers=dom,tablefmt='orgtbl'))

def transposeGraph(G):
    GT = dict((v, []) for v in G)
    for v in G:
        for w in G[v]:
            if w in GT:
                GT[w].append(v)
            else:
                GT[w] = [v]
    return GT


# From Week 2 Tutorial
def estProbTable(data, var_name, parent_names, outcomeSpace):
    """
    Calculate a dictionary probability table by ML given
    `data`, a dictionary or dataframe of observations
    `var_name`, the column of the data to be used for the conditioned variable and
    `var_outcomes`, a tuple of possible outcomes for the conditiona varible and
    `parent_names`, a tuple of columns to be used for the parents and
    `parent_outcomes` a tuple of all possible parent outcomes 
    Return a dictionary containing an estimated conditional probability table.
    """    
    var_outcomes = outcomeSpace[var_name]
    parent_outcomes = [outcomeSpace[var] for var in (parent_names)]
    # cartesian product to generate a table of all possible outcomes
    all_parent_combinations = product(*parent_outcomes)

    prob_table = odict()
    
    for i, parent_combination in enumerate(all_parent_combinations):
        cond_array = []
        parent_vars = dict(zip(parent_names, parent_combination))
        parent_index = allEqualThisIndex(data, **parent_vars)
        for var_outcome in var_outcomes:
            var_index = (np.asarray(data[var_name])==var_outcome)
            prob_table[tuple(list(parent_combination)+[var_outcome])] = (var_index & parent_index).sum()/parent_index.sum() 
           
    return {'dom': tuple(list(parent_names)+[var_name]), 'table': prob_table}

#==========================================================



In [300]:

graph = {
    'LymphNodes': [],
    'Metastasis': ['LymphNodes'],
    'BC': ['Metastasis','MC', 'SkinRetract', 'NippleDischarge', 'AD'],
    'MC': [],
    'Age': ['BC'],
    'Location': ['BC'],
    'BreastDensity': ['Mass'],
    'Mass': ['Size', 'Shape', 'Margin'],
    'Size': [],
    'Shape': [],
    'Margin': [],
    'Spiculation': ['Margin'],
    'FibrTissueDev': ['Spiculation', 'NippleDischarge', 'SkinRetract'],
    'NippleDischarge': [],
    'SkinRetract' : [],
    'AD' : ['FibrTissueDev'],
}

graphT = transposeGraph(graph)

"""
Read the data, and return an outcomeSpace dictionary with
all of the different nodes, and their domains
"""
def getOutcomeSpace(data):
    
    nodes = []
    outcomes = []
    
    for x in data:
        nodes.append(x)
        count = 0
        diffList = []
        for val in data[x]:            
            if val not in diffList:
                count += 1
                diffList.append(val)        
        outcomes.append(diffList)
        
    outcomeSpace = {}
    for i in range(len(nodes)):
        outcomeSpace[nodes[i]] = tuple(outcomes[i])
    
   
    return dict(outcomeSpace)

def learn_bayes_net(graph, file, outcomeSpace, prob_tables):
    

    with open(file) as h:
        data = pd.read_csv(h)

    # possible outcomes, by variable
    outcomeSpace = getOutcomeSpace(data)
      

    prob_tables = odict()
    for node, parents in graphT.items():    
        prob_tables[node] = estProbTable(         # Estimate the probability for a single table. 1 line
            data,
            node,
            parents,
            outcomeSpace)

    ##############################
    # Test code
    ##############################
    print('estimated P(Location)=')
    printFactor(prob_tables['Shape'])
    print()
    
  
    return outcomeSpace, prob_tables

    
prob_tables = []
outcomeSpace = []

outcomeSpace, prob_tables = learn_bayes_net(graph, 'bc 2.csv', outcomeSpace, prob_tables)


estimated P(Location)=
| Mass   | Shape     |        Pr |
|--------+-----------+-----------|
| No     | Other     | 1         |
| No     | Oval      | 0         |
| No     | Round     | 0         |
| No     | Irregular | 0         |
| Benign | Other     | 0.0553152 |
| Benign | Oval      | 0.239184  |
| Benign | Round     | 0.652967  |
| Benign | Irregular | 0.052534  |
| Malign | Other     | 0         |
| Malign | Oval      | 0.153493  |
| Malign | Round     | 0.104907  |
| Malign | Irregular | 0.7416    |



In [299]:
import random

# Recursively find child nodes such that nodes first in the ordering have no more unvisited children.
def topologicalSortRec(G, v, ordering, visited):
    
    visited.append(v)
    for child in G[v]:  
        if child not in visited:
            topologicalSortRec(G, child, ordering, visited)
        
    ordering.insert(0, v)

#Find a topological ordering on the graph
def topologicalSort(graph):
    
    ordering = []
    visited = []
    
    for node in graph:
        if node not in visited:
            topologicalSortRec(graph, node, ordering, visited)
            
           
    return ordering  # return the stack - ordering on the graph
        

def getSampleSpace(table, node):
    
    lst = list(table.items())
    
    space = list(lst[1])
    space = list(space[1].items())
    
     
    return space
    

def sampleValue(sampleSpace):
    
    rnd = random.random()
    
    lst = [0]
    
    for row in sampleSpace:
        lst.append(row[1])
        
    lst.append(1)
    lst.sort() 
    
    prob = 0
    # sampling correctly ?????
    for index in range(1, len(lst)):
        
        if rnd >= lst[index-1] and rnd < lst[index]:
            prob = lst[index]
            if prob == 1:
                prob = lst[index- 1]
            break
         
    chosenVal = ''

    for val in sampleSpace:
        if val[1] == prob:
            chosenVal = val[0]
       
                   
    return chosenVal
    
        
    
def sample(graph, prob_tables):
    
    ordering = topologicalSort(graph)
    samples = {}
    
    for node in ordering:
        sampleSpace = getSampleSpace(prob_tables[node], node)
        
        val = sampleValue(sampleSpace)
        samples[node] = val     

    return samples

#for x in range(1000):
s = sample(graph, prob_tables)
    #print(s)

('high',)
('medium',)
('low',)
[0, 0.19945, 0.30145, 0.4991, 1]
('high', 'No')
('high', 'Benign')
('high', 'Malign')
('medium', 'No')
('medium', 'Benign')
('medium', 'Malign')
('low', 'No')
('low', 'Benign')
('low', 'Malign')
[0, 0.1413888192529456, 0.16259266680024043, 0.17172223614941087, 0.17399237021064853, 0.19895812462432377, 0.24315806933156411, 0.5828495604577874, 0.6384492085754357, 0.6868889445976435, 1]
('No', 'Other')
('No', 'Oval')
('No', 'Round')
('No', 'Irregular')
('Benign', 'Other')
('Benign', 'Oval')
('Benign', 'Round')
('Benign', 'Irregular')
('Malign', 'Other')
('Malign', 'Oval')
('Malign', 'Round')
('Malign', 'Irregular')
[0, 0.0, 0.0, 0.0, 0.0, 0.05253399258343634, 0.055315203955500616, 0.10490693739424704, 0.15349286922890984, 0.2391841779975278, 0.6529666254635352, 0.7416001933768431, 1.0, 1]
('No', '<1cm')
('No', '1-3cm')
('No', '>3cm')
('Benign', '<1cm')
('Benign', '1-3cm')
('Benign', '>3cm')
('Malign', '<1cm')
('Malign', '1-3cm')
('Malign', '>3cm')
[0, 0.0, 0

In [296]:
def classify():
    return 0